In [2]:
import numpy as np
def b_dense(x_inp,w,b,activation):
    x_inp=np.array(x_inp).reshape(1,-1)
    w=np.array(w).T
    b=np.array(b).reshape(1,-1)
    Z=np.matmul(x_inp,w)+b

    return activation(Z),Z

In [3]:
import math
def b_sigmoid(x_inp,diff=False):
    def singleSigmoid(z1):
        return (1/(1+math.exp(-z1)))
    def derivative(z1):
        temp1=singleSigmoid(z1)
        return (temp1*(1-temp1))
    if diff:
        return np.vectorize(derivative)(x_inp)
    else:
        return np.vectorize(singleSigmoid)(x_inp) #https://stackoverflow.com/a/62865571
def b_linear(x_inp):
    return x_inp
def b_threshold(x_inp):
    return np.vectorize((lambda a:1 if a>0 else 0))(x_inp)

In [4]:
def b_sequential(x_inp,Weights,Biases,Activation):
    f_x=x_inp
    Zs=[]
    for i in range(len(Activation)):
        f_x,z_x=(b_dense)(f_x,Weights[i],Biases[i],Activation[i])
        #print(f'Layer {i+1}: {f_x}')
        Zs.append(z_x)
    return f_x,Zs

In [5]:
np.array([1,2,3]).reshape(1,-1)

array([[1, 2, 3]])

In [6]:
def XOR(x_inp):
    output,layers=b_sequential(x_inp,[[[1,1],[-1,-1]],[[1,1]]],[[-0.5,1.5],[-1.5]],[b_threshold for i in range(2)])
    return output,layers

In [7]:
output0,layers0=XOR([1,0])
print(layers0)
print(output0)

[array([[0.5, 0.5]]), array([[0.5]])]
[[1]]


In [8]:
def Vid(x_inp):
    output=b_sequential(x_inp,[[[0.15,0.2],[0.25,0.3]],[[0.4,0.45],[0.5,0.55]]],[[0.35,0.35],[0.6,0.6]],[b_sigmoid for i in range(2)])
    return output
o1,k1=(Vid([[0.05,0.1]]))
print(o1)

[[0.75136507 0.77292847]]


In [9]:
def b_error(output,target):
    return 0.5*sum([(target[i]-output[i])**2 for i in range(len(target))])

In [10]:
def updateVal(a,b,step):
    return [list(ai + step * bi for ai, bi in zip(sub_a, sub_b)) for sub_a, sub_b in zip(a, b)]

In [47]:
def b_backprop(x_inps,Weights,Biases,Activation,Targets,lambda_):
    BiasesOutput=[np.array(b00) for b00 in Biases]
    WeightsOutput=[np.array(w00) for w00 in Weights]

    TrainSize=len(Targets)
    for gradientDescent in range(10000):
        
        cost=0
        delCdelB=[np.zeros(len(Biases[b00])) for b00 in range(len(Biases))]
        delCdelW=[np.array(w00)*0 for w00 in Weights]
        for i in range(TrainSize):
            Weights=WeightsOutput
            Biases=BiasesOutput

            outputi,layersi=b_sequential(x_inps[i],Weights,Biases,Activation)
            cost+= b_error(outputi[0],Targets[i])
            delCdelzcurr=[((outputi[0]-Targets[i])*Activation[-1](layersi[-1][0],diff=True))]
            for ll in range(len(Biases)-1,0,-1):
                nextLayers=np.array(delCdelzcurr[0])
                outWeights=np.array(Weights[ll])
                delCdelzcurr.insert(0,np.matmul(nextLayers,outWeights)*Activation[ll](layersi[ll][0],diff=True))
            #print(delCdelzcurr)

            delCdelwcurr=[]
            for j in range(len(Weights)):
                if j==0:
                    delCdelwcurr.append(np.matmul(np.array([delCdelzcurr[j]]).T,np.array([x_inps[i]])))
                else:
                    delCdelwcurr.append(np.matmul(np.array([delCdelzcurr[j]]).T,np.array([Activation[j-1](layersi[j-1][0],diff=True)])))

            #print(delCdelwcurr)

            delCdelB=[delCdelB[b00]+delCdelzcurr[b00] for b00 in range(len(Biases))]
            delCdelW=[delCdelW[w00]+delCdelwcurr[w00] for w00 in range(len(Weights))]
            #
        cost=cost/TrainSize
        print(f"Cost at #{gradientDescent}: {cost}\r",end='')
        if gradientDescent%1000==0:
            print(f"Cost at #{gradientDescent}: {cost}")


        BiasesOutput=[BiasesOutput[b00]-(lambda_/TrainSize)*delCdelB[b00] for b00 in range(len(Biases))]
        WeightsOutput=[WeightsOutput[b00]-(lambda_/TrainSize)*delCdelW[b00] for b00 in range(len(Biases))]

    print(BiasesOutput)
    print(WeightsOutput)
    return WeightsOutput,BiasesOutput
            





print(b_backprop([[0.05,0.1]],[[[0.15,0.2],[0.25,0.3]],[[0.4,0.45],[0.5,0.55]]],[[0.35,0.35],[0.6,0.6]],[b_sigmoid for i in range(2)],[[0.01, 0.99]],0.5))

Cost at #0: 0.2983711087600027
Cost at #1000: 0.0005293050647483635
Cost at #2000: 0.00020234929960956187
Cost at #3000: 0.00010966681148919185
Cost at #4000: 6.87534279711634e-0551
Cost at #5000: 4.6740499485988055e-05
Cost at #6000: 3.3466458714951895e-05
Cost at #7000: 2.484331084615929e-055
Cost at #8000: 1.893952730947949e-055
Cost at #9000: 1.4736785000499719e-05
[array([0.45012298, 0.42603668]), array([-3.58148843,  2.95694113])]
[array([[0.15500615, 0.2100123 ],
       [0.25380183, 0.30760367]]), array([[-0.60524948, -0.55476091],
       [ 1.0638736 ,  1.11423444]])]
([array([[0.15500615, 0.2100123 ],
       [0.25380183, 0.30760367]]), array([[-0.60524948, -0.55476091],
       [ 1.0638736 ,  1.11423444]])], [array([0.45012298, 0.42603668]), array([-3.58148843,  2.95694113])])


In [ ]:
a=([[1],[2,3]])
updateVal(0*a,a,1)

[]

In [ ]:
abc=np.array([[1,2],[3,4],[5,6]])*0
abc

array([[0, 0],
       [0, 0],
       [0, 0]])

In [57]:
import random
def b_weightsbiasesrandom(layerinfo):
    brandom=[[random.uniform(-1, 1) for _ in range(num0)] for num0 in layerinfo[1:]]
    wrandom=[np.array([random.uniform(-1, 1) for _ in range(layerinfo[i]*layerinfo[i+1])]).reshape(layerinfo[i+1],layerinfo[i]) for i in range(len(layerinfo)-1)]
    return wrandom,brandom

In [59]:
xorw,xorb=b_weightsbiasesrandom((2,2,1))

xorw,xorb=b_backprop([[0,0],[0,1],[1,0],[1,1]],[[[1,1],[-1,-1]],[[1,1]]],[[-0.5,1.5],[-1.5]],[b_sigmoid for i in range(2)],[[0],[1],[1],[0]],0.001)
xorw,xorb=b_backprop([[0,0],[0,1],[1,0],[1,1]],xorw,xorb,[b_sigmoid for i in range(2)],[[0],[1],[1],[0]],0.001)

Cost at #0: 0.12438359156659173
Cost at #1000: 0.12388422204600802
Cost at #2000: 0.12346237458311114
Cost at #3000: 0.12310695273618025
Cost at #4000: 0.12280815867048582
Cost at #5000: 0.12255743059614749
Cost at #6000: 0.12234735101602905
Cost at #7000: 0.12217153967793987
Cost at #8000: 0.12202454122752426
Cost at #9000: 0.12190171436909819
[array([-0.47165502,  1.52833906]), array([-1.39079165])]
[array([[ 1.01400133,  1.01400133],
       [-0.98600157, -0.98600157]]), array([[1.04928982, 1.0484935 ]])]
Cost at #0: 0.12179912686800476
Cost at #1000: 0.12171345889403523
Cost at #2000: 0.12164191590194334
Cost at #3000: 0.12158215135858112
Cost at #4000: 0.12153219905393729
Cost at #5000: 0.12149041438946648
Cost at #6000: 0.12145542385353081
Cost at #7000: 0.12142608181879892
Cost at #8000: 0.12140143379200143
Cost at #9000: 0.12138068528531563
[array([-0.46318335,  1.53679972]), array([-1.35971274])]
[array([[ 1.01790163,  1.01790163],
       [-0.98210626, -0.98210626]]), array([[1

In [72]:
xorw,xorb=b_backprop([[0,0],[0,1],[1,0],[1,1]],xorw,xorb,[b_sigmoid for i in range(2)],[[0],[1],[1],[0]],0.001)

Cost at #0: 0.12122192879622562
Cost at #1000: 0.12122131048721949
Cost at #2000: 0.12122072170809328
Cost at #3000: 0.12122016247617534
Cost at #4000: 0.12121963281461863
Cost at #5000: 0.12121913275156979
Cost at #6000: 0.12121866231947581
Cost at #7000: 0.12121822155450226
Cost at #8000: 0.12121781049604417
Cost at #9000: 0.12121742918631315
[array([-0.4913363 ,  1.50884128]), array([-1.4534384])]
[array([[ 1.00203695,  1.00203695],
       [-0.99786346, -0.99786346]]), array([[1.2186274 , 1.20863723]])]


In [73]:
out00,layer00=b_sequential([1,0],xorw,xorb,[b_sigmoid for i in range(2)])
print(out00)

[[0.51589699]]


In [56]:
np.array([1,2]).reshape(1,2)

array([[1, 2]])